In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from PIL import Image
from collections import Counter
from wordcloud import wordcloud, STOPWORDS, ImageColorGenerator


In [2]:
conversation='WhatsApp Chat with Sudeep Pramanik.txt'

In [3]:
def date_time(s):
  pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
  result =re.match(pattern,s)
  if result:
    return True
  return False

In [4]:
def find_author(s):
  s=s.split(":")
  if len(s)==2:
    return True
  else:
    return False

In [5]:
def messages(line):
  splitline = line.split(' - ')
  datetime=splitline[0]
  date, time =datetime.split(",")
  message= " ".join(splitline[1:])

  if find_author(message):
    splitmessage=message.split(": ")
    author= splitmessage[0]
    message= " ".join(splitmessage[1:])
  else:
    author=None

  return date, time, author, message

Praparing the dataset

In [6]:

data=[]
conversation='WhatsApp Chat with Sudeep Pramanik.txt'
with open(conversation,encoding = "utf-8") as fp:
  fp.readline()
  messageBuffer=[]
  date, time, author =None, None, None

  while True:
    line=fp.readline()
    if not line:
      break
    line=line.strip()
    if date_time(line):
      if len(messageBuffer)>0:
        date.append([date,time,author,''.join(messageBuffer)])
      messageBuffer.clear()
      date,time,author,message= messages(line)
      messageBuffer.append(message)
    else:
      messageBuffer.append(line)

In [8]:
df= pd.DataFrame(data, columns=['Date','Time','Author','Message'])
df['Date']=pd.to_datetime(df['Date'])
df.head()
data=df.dropna()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid=SentimentIntensityAnalyzer()

positive_scores = []
negative_scores = []
neutral_scores = []

positive_scores = [sid.polarity_scores(i)["pos"] for i in data["Message"]]
negative_scores = [sid.polarity_scores(i)["neg"] for i in data["Message"]]
neutral_scores = [sid.polarity_scores(i)["neu"] for i in data["Message"]]

data["Positive"] = positive_scores
data["Negative"] = negative_scores
data["Neutral"] = neutral_scores

print(data.head(10))

Empty DataFrame
Columns: [Date, Time, Author, Message, Positive, Negative, Neutral]
Index: []


In [9]:
x=sum(data["Positive"])
y=sum(data["Negative"])
z=sum(data["Neutral"])

def score(A,B,C):
  if(A>B and A>C):
    print("Positive")
  elif(B>A and B>C):
    print("Negative")
  else:
    print("Neutral")

score(x,y,z)

Neutral
